In [ ]:
from seq2seq import *
import tltk, jiwer
import pandas as pd

In [ ]:
### load data
train_x = open('data/train_x.txt').read().splitlines()
train_y = open('data/train_y.txt').read().splitlines()
test_x = open('data/test_x.txt').read().splitlines()
test_y = open('data/test_y.txt').read().splitlines()

### additional dict data
dic_x = open('data/dic_x.txt').read().splitlines()
dic_y = open('data/dic_y.txt').read().splitlines()

In [ ]:
### Instantiation
data = Data()
data.set_train(train_x, train_y)
data.set_test(test_x, test_y)

### set dict data
data.set_train(dic_x, dic_y)

In [ ]:
### build no attention model
seq2seq1 = Seq2Seq(data, attention=False)
seq2seq1.build(show_summary=True)

### build attention model
seq2seq2 = Seq2Seq(data, attention=True)
seq2seq2.build(show_summary=True)

In [ ]:
### train
seq2seq1.train(batch_size=100, epoch=50, dev_split=0)
seq2seq2.train(batch_size=100, epoch=50, dev_split=0)

In [ ]:
pred1 = seq2seq1.predict(join_by='')
pred2 = seq2seq2.predict(join_by='')
df = pd.DataFrame({'thai':test_x,'pred1':pred1,'pred2':pred2, 'answer':test_y})
pred_tltk = [tltk.nlp.th2roman(x).replace('<s/>','').replace(' ','') for x in list(df.answer)]
df['tltk'] = pred_tltk
df[['thai','answer','tltk','pred1','pred2']].to_csv('pred/pred_nodict.csv', index=False)

In [ ]:
def evaluate(df):
    cols = df.columns
    gold_list = list(df.answer)
    tempdf = pd.DataFrame(index=['WER','CER macro', 'CER micro'])

    for col in cols:
        if col in ['answer', 'thai']:
            continue
        pred_list = list(df[col])
        WER = jiwer.wer(pred_list, gold_list)
        CERmacro = np.mean([jiwer.cer(p,g) for p,g in zip(pred_list, gold_list)])
        CERmicro = sum([jiwer.cer(p,g)*len(g) for p,g in zip(pred_list, gold_list)])/sum([len(g) for g in gold_list])
        tempdf[col] = [WER, CERmacro, CERmicro]
    return tempdf

evaluate(df)

### coding test

In [ ]:
i = np.random.randint(0,810)

print(test_x[i])
print(seq2seq1.predict_one(test_x[i], is_beam=True, beam_return_all=True, join_by=''))
print(test_y[i])

In [ ]:
seq2seq2.model.summary()

In [ ]:
b = np.array([1,2,3,4,5])
b[[1,4]]